In [11]:
import pandas as pd
import unidecode

In [1]:
from formula_one import FormulaOne

In [9]:
f1 = FormulaOne()
f1

Data
    Races
        (144, 7)
    Circuits
        (27, 7)
    Results
        (2947, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

In [8]:
fantasy_rosters = pd.read_csv('./draft/fantasy_rosters.csv', index_col=0)
fantasy_rosters

,OverallPick,TeamPick,Team,LastName,FirstName,Car,Nationality
0,1,1,Carlaniel & the Romeos,Norris,Lando,MCLAREN MERCEDES,GBR
1,2,1,Scuderia Spaghetti,Hamilton,Lewis,MERCEDES,GBR
2,3,1,3 Orange Whips,Verstappen,Max,RED BULL RACING HONDA,NED
3,4,1,Deep Fried,Bottas,Valtteri,MERCEDES,FIN
4,5,2,Deep Fried,Perez,Sergio,RED BULL RACING HONDA,MEX
5,6,2,3 Orange Whips,Gasly,Pierre,ALPHATAURI HONDA,FRA
6,7,2,Scuderia Spaghetti,Leclerc,Charles,FERRARI,MON
7,8,2,Carlaniel & the Romeos,Sainz,Carlos,FERRARI,ESP
8,9,3,Carlaniel & the Romeos,Ricciardo,Daniel,MCLAREN MERCEDES,AUS
9,10,3,Scuderia Spaghetti,Russell,George,WILLIAMS MERCEDES,GBR


In [15]:
f1.drivers['LastName'].apply(lambda name: unidecode.unidecode(name))

0         Aitken
1          Albon
2         Alonso
3         Bottas
4        Hartley
5         Button
6        Chilton
7       Ericsson
8          Gasly
9     Giovinazzi
10      Grosjean
11     Gutierrez
12      Hamilton
13      Haryanto
14    Hulkenberg
15        Palmer
16       Bianchi
17     Magnussen
18     Kobayashi
19        Kubica
20         Kvyat
21        Latifi
22       Leclerc
23      Lotterer
24     Maldonado
25         Massa
26    Verstappen
27       Mazepin
28         Merhi
29    Schumacher
30          Nasr
31        Norris
32          Ocon
33         Perez
34    Fittipaldi
35     Raikkonen
36      di Resta
37     Ricciardo
38       Rosberg
39         Rossi
40       Russell
41         Sainz
42      Sirotkin
43       Stevens
44        Stroll
45         Sutil
46       Tsunoda
47     Vandoorne
48        Vergne
49        Vettel
50      Wehrlein
Name: LastName, dtype: object